In [1]:
import random
import numpy as np

import nltk
import gensim.downloader as api

import torch
import torch.nn as nn
import datasets

In [3]:
dataset = datasets.load_dataset("ag_news")
dataset["train"]

Using custom data configuration default


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})

In [4]:
dataset["test"][0]

{'text': "Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.",
 'label': 2}

In [5]:
print("\n".join(api.info()['models'].keys()))

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [6]:
word2vec = api.load("glove-twitter-50")

[=====---------------------------------------------] 11.7% 23.4/199.5MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[=================---------------------------------] 35.4% 70.6/199.5MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[=============================---------------------] 59.6% 118.9/199.5MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[=========================================---------] 83.2% 166.1/199.5MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 199.5/199.5MB downloaded


In [7]:
word2vec['alaska']

array([-0.093819, -0.38502 , -0.08292 , -0.55912 , -1.0564  ,  0.21713 ,
        0.25867 , -0.25095 ,  0.58058 , -0.53596 ,  0.027442, -0.49675 ,
       -2.3067  ,  0.26241 ,  0.94936 , -0.22972 ,  0.43156 ,  0.30985 ,
        0.837   ,  0.25471 ,  0.97212 , -0.4653  , -0.37191 , -1.1439  ,
        0.22666 , -0.59356 , -0.48548 ,  0.080302,  0.11148 , -0.21289 ,
        0.52485 ,  0.14826 ,  0.21012 , -0.42637 ,  0.21639 , -0.35195 ,
       -0.0693  ,  0.76786 , -0.057718,  0.33779 , -0.27207 , -0.54431 ,
       -0.041565,  0.17028 , -0.035933,  1.007   , -0.34321 ,  0.21722 ,
       -0.32818 ,  1.644   ], dtype=float32)

In [8]:
MAX_LENGTH=128

tokenizer = nltk.WordPunctTokenizer()

dataset = dataset.map(
    lambda item: {
        "tokenized": tokenizer.tokenize(item["text"])[:MAX_LENGTH]
    }
)

  0%|          | 0/120000 [00:00<?, ?ex/s]

  0%|          | 0/7600 [00:00<?, ?ex/s]

In [9]:
word2idx = {word: idx for idx, word in enumerate(word2vec.index_to_key)}

In [10]:
def encode(word):
    if word in word2idx.keys():
        return word2idx[word]
    return word2idx["unk"]

In [11]:
dataset = dataset.map(
    lambda item: {
        "features": [encode(word) for word in item["tokenized"]]
    }
)

  0%|          | 0/120000 [00:00<?, ?ex/s]

  0%|          | 0/7600 [00:00<?, ?ex/s]

In [12]:
dataset['train'][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2,
 'tokenized': ['Wall',
  'St',
  '.',
  'Bears',
  'Claw',
  'Back',
  'Into',
  'the',
  'Black',
  '(',
  'Reuters',
  ')',
  'Reuters',
  '-',
  'Short',
  '-',
  'sellers',
  ',',
  'Wall',
  'Street',
  "'",
  's',
  'dwindling',
  '\\',
  'band',
  'of',
  'ultra',
  '-',
  'cynics',
  ',',
  'are',
  'seeing',
  'green',
  'again',
  '.'],
 'features': [62980,
  62980,
  1,
  62980,
  62980,
  62980,
  62980,
  13,
  62980,
  17,
  62980,
  20,
  62980,
  28,
  62980,
  28,
  49286,
  4,
  62980,
  62980,
  48,
  137,
  214902,
  370,
  1645,
  39,
  8606,
  28,
  380053,
  4,
  70,
  1321,
  1745,
  389,
  1]}

In [13]:
dataset = dataset.remove_columns(["text", "tokenized"])

In [14]:
def collate_fn(batch):
    max_len = max(len(row["features"]) for row in batch) #get max lenght
    input_embeds = torch.empty((len(batch), max_len), dtype=torch.long)
    labels = torch.empty(len(batch), dtype=torch.long)
    for idx, row in enumerate(batch):
        to_pad = max_len - len(row["features"])
        input_embeds[idx] = torch.cat((row["features"], torch.zeros(to_pad)))
        labels[idx] = row["label"]
    return {"features": input_embeds, "labels": labels}

In [15]:
dataset.set_format(type='torch')

In [16]:
from torch.utils.data import DataLoader

loaders = {
    k: DataLoader(
        ds, shuffle=(k=="train"), batch_size=64, collate_fn=collate_fn
    ) for k, ds in dataset.items()
}

In [17]:
dataset['train'].unique('label')

[2, 3, 1, 0]

In [18]:
class CNNModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=4):
        super().__init__()
        self.embeddings = nn.Embedding(len(word2idx), embedding_dim=embed_size)
        self.cnn = nn.Sequential(
            nn.Conv1d(embed_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1),
            nn.Flatten(),
        )
        self.cl = nn.Sequential(
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        x = self.embeddings(x)  # (batch_size, seq_len, embed_dim)
        x = x.permute(0, 2, 1)
        x = self.cnn(x)
        prediction = self.cl(x)
        return prediction

In [45]:
device = "cuda"
model = CNNModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 2

In [47]:
with torch.no_grad():
    for word, idx in word2idx.items():
        if word in word2vec:
            model.model.embeddings.weight[idx] = torch.tensor(word2vec.get_vector(word))

In [48]:
from tqdm.notebook import tqdm, trange

def training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm=2):
    for e in trange(num_epochs, leave=False):
        model.train()
        num_iter = 0
        pbar = tqdm(loaders['train'], leave=False)
        for batch in pbar:
            optimizer.zero_grad()
            input_embeds = batch['features'].to(device)
            labels = batch['labels'].to(device)
            prediction = model(input_embeds)
            loss = criterion(prediction, labels)
            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            num_iter += 1
        valid_loss = 0
        valid_acc = 0
        num_iter = 0
        model.eval()
        with torch.no_grad():
            correct = 0
            num_objs = 0
            for batch in loaders["test"]:
                input_embeds = batch["features"].to(device)
                labels = batch["labels"].to(device)
                prediction = model(input_embeds)
                valid_loss += criterion(prediction, labels)
                correct += (labels == prediction.argmax(-1)).float().sum()
                num_objs += len(labels)
                num_iter += 1

        print(f"Valid Loss: {valid_loss / num_iter}, accuracy: {correct/num_objs}")
            

In [49]:
model

CNNModel(
  (embeddings): Embedding(1193514, 50)
  (cnn): Sequential(
    (0): Conv1d(50, 50, kernel_size=(3,), stride=(2,), padding=(1,))
    (1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv1d(50, 50, kernel_size=(3,), stride=(2,), padding=(1,))
    (4): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv1d(50, 50, kernel_size=(3,), stride=(2,), padding=(1,))
    (7): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): AdaptiveMaxPool1d(output_size=1)
    (10): Flatten(start_dim=1, end_dim=-1)
  )
  (cl): Sequential(
    (0): Linear(in_features=50, out_features=4, bias=True)
  )
)

In [50]:
training(model, criterion, optimizer, num_epochs, loaders)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1875 [00:00<?, ?it/s]

Valid Loss: 0.3699994385242462, accuracy: 0.8731578588485718


  0%|          | 0/1875 [00:00<?, ?it/s]

Valid Loss: 0.35893747210502625, accuracy: 0.8739473223686218


In [51]:
class RNN(nn.Module):
    def __init__(self, embedding_size, hidden_size):
        super().__init__()
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        
        self.w_x = nn.Parameter(torch.randn(embedding_size, hidden_size))
        self.w_h = nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.b_x = nn.Parameter(torch.randn(1, hidden_size))
        self.b_h = nn.Parameter(torch.randn(1, hidden_size))
        
    def forward(self, x, hidden=None):
        if hidden is None:
            hidden = torch.zeros(x.size(0), self.hidden_size).to(x.device)
        seq_len = x.size(1)
        for idx in range(seq_len):
            hidden = torch.tanh(
                x[:, idx] @ self.w_x + self.b_x + hidden @ self.w_h + self.b_h
            )
        return hidden
        

In [52]:
class RNNModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=4):
        super().__init__()
        self.embeddings = nn.Embedding(len(word2idx), embed_size)
        self.rnn = RNN(embed_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embeddings(x)
        hidden = self.rnn(x)
        output = self.cls(hidden)
        return output

In [54]:
device = "cuda" if torch.cuda.is_available() else "cpu"

rnn_model = RNNModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn_model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [55]:
training(rnn_model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1875 [00:00<?, ?it/s]

Valid Loss: 1.4551663398742676, accuracy: 0.2567105293273926


In [56]:
class TorchRNNModel(nn.Module):
    def __init__(self, embedding_size, hidden_size, num_classes=4):
        super().__init__()
        self.hidden_size = hidden_size
        
        self.embeddings = nn.Embedding(len(word2idx), embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, batch_first=True)
        self.clf = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        x = self.embeddings(x)
        hidden = self.rnn(x)
        output = self.clf(hidden[1].squeeze(0))
        return output        

In [57]:
device = "cuda" if torch.cuda.is_available() else "cpu"

torch_rnn_model = TorchRNNModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn_model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

training(torch_rnn_model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1875 [00:00<?, ?it/s]

Valid Loss: 1.4159088134765625, accuracy: 0.2505263090133667


In [58]:
r = nn.RNN(10, 20, batch_first=True)
input = torch.randn(5, 10)
#h0 = torch.randn(2, 3, 20)
output, hn = r(input)

In [59]:
output.size(), hn.size()

(torch.Size([5, 20]), torch.Size([1, 20]))

In [60]:
class GRU(nn.Module):
    def __init__(self, embed_size, hidden_size):
        super().__init__()

        self.embed_size = embed_size
        self.hidden_size = hidden_size

        self.w_rh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_rh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_rx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_rx = nn.Parameter(torch.rand(1, hidden_size))

        self.w_zh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_zh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_zx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_zx = nn.Parameter(torch.rand(1, hidden_size))

        self.w_nh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_nh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_nx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_nx = nn.Parameter(torch.rand(1, hidden_size))

    def forward(self, x, hidden = None):
        '''
        x – torch.FloatTensor with the shape (bs, seq_length, emb_size)
        hidden - torch.FloatTensro with the shape (bs, hidden_size)
        return: torch.FloatTensor with the shape (bs, hidden_size)
        '''
        if hidden is None:
            hidden = torch.zeros((x.size(0), self.hidden_size)).to(x.device)
        
        for cur_idx in range(x.size(1)):
            r = torch.sigmoid(
                x[:, cur_idx] @ self.w_rx + self.b_rx + hidden @ self.w_rh + self.b_rh
            )
            z = torch.sigmoid(
                x[:, cur_idx] @ self.w_zx + self.b_zx + hidden @ self.w_zh + self.b_zh
            )
            n = torch.tanh(
                x[:, cur_idx] @ self.w_nx + self.b_nx + r * (hidden @ self.w_nh + self.b_nh)
            )
            hidden = (1 - z) * n + z * hidden

        return hidden

In [61]:
class GRUModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=4):
        super().__init__()
        self.embed = nn.Embedding(len(word2idx), embed_size)
        self.gru = GRU(embed_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embed(x)
        hidden = self.gru(x)
        output = self.cls(hidden)
        return output

In [62]:
gru_model = GRUModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru_model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [63]:
training(gru_model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1875 [00:00<?, ?it/s]

Valid Loss: 0.5521388053894043, accuracy: 0.8010525703430176


In [64]:
device = "cuda" if torch.cuda.is_available() else "cpu"

gru_emb_model = GRUModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru_emb_model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [65]:
gru_emb_model.embed.weight[0]

tensor([-1.7579e+00, -7.3016e-01,  1.5156e+00,  8.3016e-03, -1.2871e+00,
        -6.6036e-01,  1.5145e-04, -1.2049e-01, -1.6718e+00,  5.7664e-01,
        -1.1666e-01,  1.8611e-01, -7.8184e-01, -1.1965e+00,  4.3965e-01,
         6.9593e-01, -2.0213e-02,  1.5535e-01, -3.3549e-01, -6.3869e-02,
         4.8957e-01,  1.3508e+00, -1.2582e+00, -2.3653e+00,  1.1004e-01,
        -2.0959e-01, -9.0654e-01, -6.9989e-01, -1.2343e+00, -1.5817e-01,
        -1.5985e+00,  1.3898e+00, -9.9889e-01,  2.4111e-01,  3.1420e-01,
         3.1689e-01,  1.1221e+00,  1.2584e+00, -1.2355e-02, -9.3369e-01,
         1.3225e+00,  1.5377e+00, -7.3059e-01,  1.1931e-01, -2.4281e+00,
        -4.5330e-01,  1.8753e+00,  2.0441e+00, -1.1219e+00,  1.7999e-01],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [66]:
with torch.no_grad():
    for word, idx in word2idx.items():
        if word in word2vec:
            gru_emb_model.embed.weight[idx] = torch.tensor(word2vec.get_vector(word))

In [67]:
gru_emb_model.embed.weight[0]

tensor([ 0.7870,  0.7215,  0.2915, -0.0565,  0.3168,  0.4717,  0.0235,  0.6957,
         0.2078,  0.6098, -0.2239,  0.7481, -2.6208,  0.2012, -0.4810,  0.1290,
         0.0352, -0.2449, -0.3609,  0.0267,  0.2898, -0.1070, -0.3462,  0.0211,
         0.5451, -1.0958, -0.2740,  0.2233,  1.0827, -0.0290, -0.8403,  0.5862,
        -0.3651,  0.3402,  0.8961,  0.3276,  0.2427,  0.6840, -0.3437,  0.1358,
        -2.2162, -0.4254,  0.4616,  0.8863, -0.2201,  0.0256, -0.3862,  0.0801,
        -0.0753, -0.6146], device='cuda:0', grad_fn=<SelectBackward0>)

In [ ]:
training(gru_emb_model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1875 [00:00<?, ?it/s]

Valid Loss: 0.47915175557136536, accuracy: 0.8349999785423279


In [ ]:
def freeze_embeddings(model, req_grad=False):
    embeddings = model.embed
    for c_p in embeddings.parameters():
        c_p.requires_grad = req_grad

In [ ]:
def training_freeze(model, criterion, optimizer, num_epochs, loaders, max_grad_norm=2, num_freeze_iter=1000):
    freeze_embeddings(model)
    for e in trange(num_epochs, leave=False):
        model.train()
        num_iter = 0
        pbar = tqdm(loaders["train"], leave=False)
        for batch in pbar:
            if num_iter > num_freeze_iter and e < 1:
                freeze_embeddings(model, True)
            optimizer.zero_grad()
            input_embeds = batch["features"].to(device)
            labels = batch["labels"].to(device)
            prediction = model(input_embeds)
            loss = criterion(prediction, labels)
            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            num_iter += 1
        valid_loss = 0
        valid_acc = 0
        num_iter = 0
        model.eval()
        with torch.no_grad():
            correct = 0
            num_objs = 0
            for batch in loaders["test"]:
                input_embeds = batch["features"].to(device)
                labels = batch["labels"].to(device)
                prediction = model(input_embeds)
                valid_loss += criterion(prediction, labels)
                correct += (labels == prediction.argmax(-1)).float().sum()
                num_objs += len(labels)
                num_iter += 1

        print(f"Valid Loss: {valid_loss / num_iter}, accuracy: {correct/num_objs}")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

gru_emb_model_1 = GRUModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru_emb_model_1.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [ ]:
with torch.no_grad():
    for word, idx in word2idx.items():
        if word in word2vec:
            gru_emb_model_1.embed.weight[idx] = torch.from_numpy(word2vec.get_vector(word))

In [73]:
training_freeze(gru_emb_model_1, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1875 [00:00<?, ?it/s]

Valid Loss: 0.4601065516471863, accuracy: 0.8417105078697205
